In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
sns.set()

In [2]:
import pickle
close_path = "/Users/mingyu/Desktop/Alpha_data/DataBase/Daily/close.pkl"
amt_path = "/Users/mingyu/Desktop/Alpha_data/DataBase/Daily/amt.pkl"
close = pd.read_pickle(close_path).astype("float64")
amt = pd.read_pickle(amt_path).astype("float64")

In [3]:
close_dropna = close.dropna(thresh=1400, axis=1)
close_dropna = close_dropna.dropna()
cols = ["BTCUSDT", "ETHUSDT"] + list(close_dropna.columns[2:])
close_dropna = close_dropna.loc[:, cols]
amt_dropna = amt.loc[close_dropna.index, cols]

In [4]:
close_dropna_val = close_dropna.values
amt_dropna_val = amt_dropna.values
time_periods, n_assets = close_dropna.shape

In [5]:
close_dropna

symbol,BTCUSDT,ETHUSDT,BNBUSDT,NEOUSDT,LTCUSDT,QTUMUSDT,ADAUSDT,XRPUSDT,EOSUSDT,IOTAUSDT,...,UMAUSDT,BELUSDT,WINGUSDT,UNIUSDT,OXTUSDT,SUNUSDT,AVAXUSDT,FLMUSDT,ORNUSDT,UTKUSDT
20201002,10570.40,345.60,27.3369,18.089,44.88,2.288,0.09286,0.23386,2.4787,0.2688,...,7.811,1.4880,24.546,3.8904,0.2715,15.78500,3.9115,0.4673,2.4947,0.17010
20201003,10542.06,345.95,28.1867,17.806,45.51,2.297,0.09328,0.23273,2.4786,0.2659,...,7.960,1.3900,24.810,3.9041,0.2700,16.03000,4.0168,0.4152,2.3919,0.13850
20201004,10666.63,352.37,28.9668,18.125,46.69,2.306,0.09686,0.24794,2.5176,0.2750,...,7.043,1.2393,19.484,3.7262,0.2626,15.80700,3.9640,0.3597,2.2399,0.13970
20201005,10792.21,353.62,28.6158,17.664,46.34,2.302,0.09738,0.25030,2.5257,0.2712,...,7.116,1.0841,19.251,3.3190,0.2666,15.78800,3.9813,0.3798,2.0754,0.13230
20201006,10599.66,340.75,27.6812,16.569,45.99,2.172,0.09289,0.24545,2.6959,0.2533,...,6.545,1.0716,21.667,2.7143,0.2456,14.77200,3.5424,0.3009,1.7936,0.11170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20240716,65043.99,3444.13,576.2000,11.340,73.21,2.567,0.43810,0.57930,0.5984,0.1710,...,2.237,0.5633,5.320,8.1740,0.0804,0.01108,28.0100,0.0698,1.6323,0.05629
20240718,63987.92,3426.50,571.9000,11.390,71.63,2.670,0.42340,0.56970,0.5872,0.1704,...,2.228,0.5540,5.290,7.8340,0.0834,0.01107,27.1400,0.0696,1.5076,0.05313
20240719,66660.00,3503.53,593.4000,11.800,73.40,2.756,0.43900,0.57330,0.6077,0.1759,...,2.440,0.5664,5.380,7.9950,0.0858,0.01073,28.1600,0.0712,1.5850,0.05545
20240720,67139.96,3517.50,593.1000,11.790,73.03,2.732,0.43730,0.59370,0.6090,0.1753,...,2.386,0.5651,5.430,8.0120,0.0847,0.01071,29.1100,0.0718,1.4881,0.05539


In [6]:
def initialization(n_assets, time_periods):

    # Random market caps and prices
    index_init = 1000
    market_caps = amt_dropna_val[0, :]
    prices = close_dropna_val
    
    # Weighting factor
    weighting_factors = market_caps / np.sum(market_caps) * 100
    
    return index_init, weighting_factors, prices

In [7]:
def simulation(index_init, weighting_factors, prices):
    # Initialize index and divisor
    indices = np.zeros(time_periods)
    weights = np.zeros((time_periods, n_assets))
    divisors = np.zeros(time_periods)
    
    # Initial weights & divisor
    weights[0] = index_init * weighting_factors / prices[0]
    divisors[0] = np.sum(prices[0] * weights[0])
    indices[0] = np.sum(prices[0] * weights[0] / divisors[0]) * 1000
    
    # Compute Indices and Divisors over time
    for t in range(1, time_periods):
        # Calculate weights for time t
        weights[t] = indices[t-1] * weighting_factors / prices[t]
        
        # Update Divisors for time t
        divisors[t] = divisors[t-1] * (np.sum(prices[t] * weights[t]) / np.sum(prices[t-1] * weights[t-1]))
        
        # Update the Index for time t
        indices[t] = np.sum(prices[0] * weights[t] / divisors[t]) * 1000
    
    return indices

In [8]:
index_init, weighting_factors, prices = initialization(n_assets, time_periods)
indices = simulation(index_init, weighting_factors, close_dropna_val)
indices_df = pd.DataFrame()
indices_df["indices"] = indices
indices_df.describe()

,indices
count,1141.000000
mean,1533.435439
std,971.446563
min,176.639796
25%,708.925773
50%,1290.524218
75%,2511.061728
max,5682.316490
